## CARE restoration and CARE projection predictions ##

We will use in this notebook CARE restoration (can be called CARE denoising also) and CARE projection trained models to perform predictionS of 2D projections of low illuminated stacks. 

It is possible to predict only the restoration of the 3D stacks, or to predict both restoration and projection. We can skip the saving of restored images if we know in advance that the projectcion will be good. 

Firstly, we import all the functions we need. We will specify the GPU we wants to use in the last line. We have to check that the GPU is not already use to avoid memory issues.

In [2]:
from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
import os
import glob


from tifffile import imread
from csbdeep.utils import Path, download_and_extract_zip_file, plot_some
from csbdeep.io import save_tiff_imagej_compatible
from csbdeep.models import CARE, ProjectionCARE
from helpers import save_8bit_tiff_imagej_compatible

try:
    from pathlib import Path
    Path().expanduser()
except (ImportError, AttributeError):
        from pathlib2 import Path

try:
        import tempfile
        tempfile.TemporaryDirectory
except (ImportError, AttributeError):
       from backports import tempfile    
        

from skimage import exposure

import time

os.environ["CUDA_VISIBLE_DEVICES"]="1"

**Processing of the movie**

All the data we want to process must be conatined in a unique folder. We will give the path to the folder through `basedir`.

- `basedirResults3D` will be the path were restored stacks will be saved,
- `basedirResults2D` wwill be the path were projected stacks will be saved,
- `basedirResults3Dextended` will give the prefix of the restored images names, 
- `basedirResults2Dextended` will give the prefix of the projected images names

`Model_Dir` is the path where all the CARE trained models are stored.

In [3]:

basedir='/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/f_bosveld/CARING/shineG4_EcadGFPki_wRNAi_24hL_1' 

basedirResults3D= basedir + '/Restored'
basedirResults2D= basedir + '/Projected'
basedirResults3Dextended= basedirResults3D + '/Restored'
basedirResults2Dextended= basedirResults2D + '/Projected'

Model_Dir='/run/media/sancere/DATA/Lucas_Model_to_use/CARE'

Here we select the model we will use for the prediction and we load it.

In [4]:
RestorationModel = 'CARE_restoration_SpinWideFRAP4_Bin2'
ProjectionModel ='CARE_projection_SpinWideFRAP4_Bin2'

RestorationModel = CARE(config = None, name = RestorationModel, basedir = Model_Dir)
ProjectionModel = ProjectionCARE(config = None, name = ProjectionModel, basedir = Model_Dir) 

Loading network weights from 'weights_best.h5'.


Then we create the `basedirResults3D` and `basedirResults2D` folders were the predictions will be stored if the folders don't exist yet. We then list all the files with the given file extension (* *choose extenstion*) in the `basedir` folder.

In [5]:
Path(basedirResults3D).mkdir(exist_ok = True)
Path(basedirResults2D).mkdir(exist_ok = True)

Raw_path = os.path.join(basedir, '*TIF') #tif or TIF be careful

axes = 'ZYX'  #projection axes : 'YX'

filesRaw = glob.glob(Raw_path)

We finally apply the predictions on each stacks found in the folder. 

The 3rd line is to avoid to make a CARE prediction (restoration or projection) on a stack that was already processed by the network before.

If the we know in advance that the model perform well for the predicted dataset, we can convert the predictions into 8bits (`.astype('uint8')`). For the creation of the training set for projection, the restored images has to be converted into 16bits. In the other cases, we won't convert the images with Python but with ImageJ or Fiji.

In [6]:
for fname in filesRaw:
     if  os.path.exists(fname) == True :
            if  os.path.exists(basedirResults3Dextended + '_' + os.path.basename(fname)) == False or os.path.exists(basedirResults2Dextended + '_' + os.path.basename(fname)) == False :
                print(fname)
                y = imread(fname)
                restored = RestorationModel.predict(y, axes, n_tiles = (1,2,4)) #n_tiles is for the decomposition of the image in (z,y,x). (1,2,2) will work with light images. Less tiles we have, faster the calculation is 
                projection = ProjectionModel.predict(restored, axes, n_tiles = (1,1,1)) #n_tiles is for the decomposition of the image in (z,y,x). There is overlapping in the decomposition wich is managed by the program itself
                axes_restored = axes.replace(ProjectionModel.proj_params.axis, '')
                restored = restored.astype('uint8') # if prediction and projection running at the same time
                #restored = restored.astype('uint16') # if projection training set creation or waiting for a future projection 
                projection = projection.astype('uint8')
                save_tiff_imagej_compatible((basedirResults3Dextended  + '_' +  os.path.basename(fname)) , restored, axes)
                save_tiff_imagej_compatible((basedirResults2Dextended + '_' + os.path.basename(fname)) , projection, axes_restored)
